# inferSent & ResNet50

## Dataloader

In [1]:
import os
import glob
from PIL import Image

from torch.utils.data import Dataset
import torchvision.transforms as transforms


class SurgicalVQADataset(Dataset):
    def __init__(self, seq, folder_head, folder_tail, labels, transform=None):
        
        self.transform = transform
        
        # files, question and answers
        filenames = []
        for curr_seq in seq: filenames = filenames + glob.glob(folder_head + str(curr_seq) + folder_tail)
        self.vqas = []
        for file in filenames:
            file_data = open(file, "r")
            lines = [line.strip("\n") for line in file_data if line != "\n"]
            file_data.close()
            for line in lines: self.vqas.append([file, line])
        print('Total files: %d | Total question: %.d' %(len(filenames), len(self.vqas)))
        
        # Labels
        self.labels = labels
        
    def __len__(self):
        return len(self.vqas)

    def __getitem__(self, idx):
        
        # img
        loc = self.vqas[idx][0].split('/')
        img_loc = os.path.join(loc[0],loc[1],loc[2], 'left_frames',loc[-1].split('_')[0]+'.png')
        img = Image.open(img_loc)
        if self.transform: img = self.transform(img)
            
        # question and answer
        question = self.vqas[idx][1].split('|')[0]
        label = self.labels.index(str(self.vqas[idx][1].split('|')[1]))

        return img, question, label

## Model Version 1

In [2]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from torchvision import models

class Surgical_VQA(nn.Module):
    def __init__(self, num_classes=12):
        super(Surgical_VQA, self).__init__()

        # text processing
        self.text_feature_extractor = SentenceTransformer('bert-base-nli-mean-tokens')
        # image processing
        self.img_feature_extractor = models.resnet50(pretrained=True)
        new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
        self.img_feature_extractor.fc = new_fc

        #classifier
        self.classifier = nn.Linear(2816, num_classes)

    def forward(self, img, text):
        img_feature = self.img_feature_extractor(img)
        
        text_feature = self.text_feature_extractor.encode(text)
        text_feature = torch.tensor(text_feature).cuda()
        
        img_text_features = torch.cat((img_feature, text_feature), dim=1)
        
        out = self.classifier(img_text_features)
        return out

## Model Version 2

In [2]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from torchvision import models

class Surgical_VQA(nn.Module):
    def __init__(self, num_classes=12):
        super(Surgical_VQA, self).__init__()

        # text processing
        self.text_feature_extractor = SentenceTransformer('bert-large-nli-mean-tokens')
        # image processing
        self.img_feature_extractor = models.resnet50(pretrained=True)
        new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
        self.img_feature_extractor.fc = new_fc

        #classifier
        self.classifier = nn.Linear(3072, num_classes)

    def forward(self, img, text):
        img_feature = self.img_feature_extractor(img)
        
        text_feature = self.text_feature_extractor.encode(text)
        text_feature = torch.tensor(text_feature).cuda()
        
        img_text_features = torch.cat((img_feature, text_feature), dim=1)
        
        out = self.classifier(img_text_features)
        return out

## Model Version 3

In [2]:
import torch
import torch.nn as nn
from torchvision import models

import nltk
nltk.download('punkt')
from InferSent.models import InferSent

class Surgical_VQA(nn.Module):
    def __init__(self, num_classes=12):
        super(Surgical_VQA, self).__init__()

        # text processing
        params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                        'pool_type': 'max', 'dpout_model': 0.0, 'version': 2}
        self.text_feature_extractor = InferSent(params_model)
        self.text_feature_extractor.load_state_dict(torch.load('InferSent/encoder/infersent2.pkl'))
        self.text_feature_extractor.set_w2v_path('InferSent/fastText/crawl-300d-2M.vec')
        self.text_feature_extractor.build_vocab_k_words(K=100000)
        
        # image processing
        self.img_feature_extractor = models.resnet50(pretrained=True)
        new_fc = nn.Sequential(*list(self.img_feature_extractor.fc.children())[:-1])
        self.img_feature_extractor.fc = new_fc

        #classifier
        self.classifier = nn.Linear(6144, num_classes)

    def forward(self, img, text):
        img_feature = self.img_feature_extractor(img)
        
        text_feature = self.text_feature_extractor.encode(text) #infersent.encode(query)[0]
        text_feature = torch.tensor(text_feature).cuda()
        
        img_text_features = torch.cat((img_feature, text_feature), dim=1)
        
        out = self.classifier(img_text_features)
        return out

[nltk_data] Downloading package punkt to /home/mobarak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Metrics

In [3]:
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score

def calc_acc(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    return acc

def calc_classwise_acc(y_true, y_pred):
    matrix = confusion_matrix(y_true, y_pred)
    classwise_acc = matrix.diagonal()/matrix.sum(axis=1)
    return classwise_acc

def calc_map(y_true, y_scores):
    mAP = average_precision_score(y_true, y_scores,average=None)
    return mAP

## Test model

In [4]:
import torch.nn.functional as F

def test_model(epoch, model, valid_dataloader):
    
    model.eval()

    total_loss = 0.0    
    label_true = None
    label_pred = None
    label_score = None
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for i, (imgs, q, labels) in enumerate(valid_dataloader, 0):
            questions = []
            for question in q: questions.append(question)
            imgs, labels = imgs.cuda(), labels.cuda()
            
            outputs = model(imgs, questions)

            loss = criterion(outputs,labels)
            total_loss += loss.item()
        
            scores, predicted = torch.max(F.softmax(outputs, dim=1).data, 1)    
            label_true = labels.data.cpu() if label_true == None else torch.cat((label_true, labels.data.cpu()), 0)
            label_pred = predicted.data.cpu() if label_pred == None else torch.cat((label_pred, predicted.data.cpu()), 0)
            label_score = scores.data.cpu() if label_score == None else torch.cat((label_score, scores.data.cpu()), 0)

            
    acc, c_acc, mAP = calc_acc(label_true, label_pred), calc_classwise_acc(label_true, label_pred), 0.0#calc_map(label_true, label_score)

    print('Test: epoch: %d loss: %.6f | Acc: %.6f | mAP: %.6f' %(epoch, total_loss, acc, mAP))
    print(c_acc)
    
    return (acc, c_acc, mAP)

## Train model

In [5]:
from torch import optim
def train_model(epoch, model, train_dataloader, lr):  # train model
    
    model.train()
    
    total_loss = 0.0    
    label_true = None
    label_pred = None
    label_score = None
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay = 0)
    
    for i, (imgs, q, labels) in enumerate(train_dataloader,0):
        questions = []
        for question in q: questions.append(question)
        imgs, labels = imgs.cuda(), labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = model(imgs, questions)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        total_loss += loss.item()
        
        scores, predicted = torch.max(F.softmax(outputs, dim=1).data, 1)    
        label_true = labels.data.cpu() if label_true == None else torch.cat((label_true, labels.data.cpu()), 0)
        label_pred = predicted.data.cpu() if label_pred == None else torch.cat((label_pred, predicted.data.cpu()), 0)
        label_score = scores.data.cpu() if label_score == None else torch.cat((label_score, scores.data.cpu()), 0)

    
    # loss and acc
    acc, c_acc, mAP = calc_acc(label_true, label_pred), calc_classwise_acc(label_true, label_pred), 0.0#calc_map(label_true, label_score)

    print('Train: epoch: %d loss: %.6f | Acc: %.6f | mAP: %.6f' %(epoch, total_loss, acc, mAP))
    return

## Main

In [6]:
import os
import torch

from torchvision import transforms
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="1"

def seed_everything(seed=27):
    '''
    Set random seed for reproducible experiments
    Inputs: seed number 
    '''
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
if __name__ == "__main__":
     
    # Set random seed
    seed_everything()  
    
    # Device Count
    num_gpu = torch.cuda.device_count()
    
    # hyperparameters
    bs = 32
    epochs = 150
    lr = 0.00001
    
    checkpoint_dir = 'checkpoints/v3/simple/'
    
    # train and test dataloader
    train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
    val_seq = [1, 5, 16]
    folder_head = 'dataset/instruments18/seq_'
    folder_tail = '/vqa/simple/*.txt'

    labels = ['kidney',
          'Idle', 'Grasping', 'Retraction', 'Tissue_Manipulation',
          'Tool_Manipulation', 'Cutting', 'Cauterization', 'Suction', 
          'Looping', 'Suturing', 'Clipping', 'Staple', 'Ultrasound_Sensing',
          'left-top', 'right-top', 'left-bottom', 'right-bottom']

    transform = transforms.Compose([
                transforms.Resize((300,256)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

    # train_dataset
    train_dataset = SurgicalVQADataset(train_seq, folder_head, folder_tail, labels, transform=transform)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size= bs, shuffle=True)

    # Val_dataset
    val_dataset = SurgicalVQADataset(val_seq, folder_head, folder_tail, labels, transform=transform)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size= bs, shuffle=False)
    
    # model
    model = Surgical_VQA(num_classes=len(labels)).cuda()
    
    best_epoch = [0]
    best_results = [0.0]
    
    for epoch in range(1, epochs):
        train_model(epoch, model, train_dataloader, lr)
        test_acc, test_c_acc, mAP = test_model(epoch, model, train_dataloader)
    
        if test_acc >= best_results[0]:
            best_results[0] = test_acc
            best_epoch[0] = epoch
        
        print('Best epoch: %d | Best acc: %.6f' %(best_epoch[0], best_results[0]))
        checkpoint = {'lr': lr, 'b_s': bs, 'state_dict': model.state_dict() }
        save_name = "checkpoint_" + str(epoch) + '_epoch.pth'
        
        torch.save(checkpoint, os.path.join(checkpoint_dir, save_name))

Total files: 1560 | Total question: 9014
Total files: 447 | Total question: 2769
Vocab size : 100000


/media/mobarak/data/lalith/SurgicalScene_VQA/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Test: epoch: 1 loss: 643.626866 | Acc: 0.261482 | mAP: 0.000000


/media/mobarak/data/lalith/SurgicalScene_VQA/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Test: epoch: 1 loss: 556.344057 | Acc: 0.353672 | mAP: 0.000000
[0.43269231 0.74832419 0.         0.         0.34653465 0.
 0.00207039 0.         0.         0.         0.         0.
 0.         0.         0.52554745 0.20345964 0.         0.        ]
Best epoch: 1 | Best acc: 0.353672


...

Test: epoch: 148 loss: 141.473929 | Acc: 0.694586 | mAP: 0.000000
[1.         0.82937233 0.89830508 0.56430446 0.61810467 0.4609375
 0.33954451 1.         1.         0.58536585 1.         0.63414634
 1.         0.47222222 0.57067021 0.69110379 0.24105461 0.76      ]
Best epoch: 137 | Best acc: 0.696250


/media/mobarak/data/lalith/SurgicalScene_VQA/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Test: epoch: 149 loss: 145.611057 | Acc: 0.669958 | mAP: 0.000000


/media/mobarak/data/lalith/SurgicalScene_VQA/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Test: epoch: 149 loss: 141.600976 | Acc: 0.695585 | mAP: 0.000000
[1.         0.84765387 0.22033898 0.12860892 0.63932107 0.28125
 0.6252588  1.         1.         0.79674797 1.         0.63414634
 1.         0.83333333 0.60384871 0.53459638 0.89642185 0.32842105]
Best epoch: 137 | Best acc: 0.696250
